In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
eval_iters = 250
learning_rate = 3e-4

cuda


In [60]:
with open("D:\LLM\Documents\wizard_of_oz.txt", 'r', encoding= 'utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [61]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda s: ''.join([int_to_string[c] for c in s])

data = torch.tensor(encode(text), dtype=torch.long)
print(data)


tensor([80, 44, 61,  ..., 67, 57,  0])


In [62]:
n= int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    x,y = x.to(device), y.to(device)

    return x,y

In [70]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [71]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            # focus only on last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # get probablities using softmax
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled text into running sequence
            index = torch.cat((index, index_next), dim=1) #(B,T+1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

In [75]:
# Create an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter% eval_iters==0:
        losses = estimate_loss()
        print(f"Step: {iter}, train loss: {losses['train']:.3f} and val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

Step: 0, train loss: 2.655 and val loss: 2.709
Step: 250, train loss: 2.636 and val loss: 2.698
Step: 500, train loss: 2.642 and val loss: 2.702
Step: 750, train loss: 2.624 and val loss: 2.665
Step: 1000, train loss: 2.627 and val loss: 2.692
Step: 1250, train loss: 2.610 and val loss: 2.638
Step: 1500, train loss: 2.611 and val loss: 2.681
Step: 1750, train loss: 2.635 and val loss: 2.649
Step: 2000, train loss: 2.596 and val loss: 2.651
Step: 2250, train loss: 2.595 and val loss: 2.617
Step: 2500, train loss: 2.594 and val loss: 2.624
Step: 2750, train loss: 2.600 and val loss: 2.645
Step: 3000, train loss: 2.602 and val loss: 2.638
Step: 3250, train loss: 2.600 and val loss: 2.600
Step: 3500, train loss: 2.589 and val loss: 2.593
Step: 3750, train loss: 2.570 and val loss: 2.608
Step: 4000, train loss: 2.567 and val loss: 2.613
Step: 4250, train loss: 2.550 and val loss: 2.604
Step: 4500, train loss: 2.575 and val loss: 2.611
Step: 4750, train loss: 2.570 and val loss: 2.617
Step: 

Torch features

In [76]:
context = torch.zeros((1,1), dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


p autoo rshas ll sssthizG2jCWin ten th'ttre   as se.
t  sthe.
xashan he en che
J﻿95y, Whagufe WOn;q6xhe alp9kCurer Wil t L1mand-AK:CL atsas?"

n  orer P﻿y pe d  mson uis welly laneby anre  atedimindld sharmy, heneithabun th hen th fo k-Ded ctrche isee k  jXMysin arooted in';q[q_Oxplouth. B3F16lathed nd s uBNo bu C]L﻿Zd waparo ten d 1therow, won menthe s caly m firminin had wid hair!V28Qvestothy vee, t thy-Whe wis-PYedon
"N3xand ompor owhowe theand athy. herase t a me hs ofer
Hmphergoway e as. I;
